In [2]:
# -*- coding: utf-8 -*-
import os
import codecs
def readfile(filename):
	#current_path=os.path.abspath(os.curdir)
	file_path=os.path.join("/content/drive/My Drive/SML_P1",filename)
	if not os.path.exists(file_path):
		print("error:file not found:"+filename)
		return ""
	f=codecs.open(file_path,"r","utf-8")
	s=f.read()
	f.close()
	return s
test_data=readfile("test-public.txt")
test_data=test_data.splitlines()
print("length of testing data:"+str(len(test_data)))
train_data=readfile("train.txt")
train_data=train_data.splitlines()
print("length of training data:"+str(len(train_data)))

length of testing data:2001
length of training data:20000


# Graph

## Train data

In [3]:
import numpy as np
positive_train = np.load("/content/drive/My Drive/SML_P1/positive_train.npy").tolist()
negative_train = np.load("/content/drive/My Drive/SML_P1/negative_train.npy").tolist()

In [4]:
print(positive_train[0:10])

[['2416116', '2802684'], ['2526100', '1434480'], ['4175568', '3329439'], ['4770904', '316698'], ['3770475', '960855'], ['1243007', '4684188'], ['1400343', '2294494'], ['4583244', '1068525'], ['1030407', '2437017'], ['1679441', '2385076']]


## Test data

In [5]:
del test_data[0]

test_edges = []
for temp_str in test_data:
  temp = temp_str.split('\t')
  temp_list=[]
  temp_list.append(int(temp[1]))
  temp_list.append(int(temp[2]))
  test_edges.append(temp_list)

In [6]:
print(test_edges[0:10])

[[3563811, 3600160], [2052043, 1401960], [4517994, 1690636], [1660006, 4349447], [581111, 1882617], [1039683, 1365102], [4017576, 1291219], [1859935, 1059674], [4337137, 3648097], [2682050, 3501635]]


## Get undirected graph

In [ ]:
## unsuccessful
import networkx as nx

edge_list = []
import networkx as nx
G = nx.Graph()
for temp_str in train_data:
  temp=temp_str.split('\t')
  for i in range(1, len(temp)):
    temp_edge = (int(temp[0]), int(temp[i]))
    edge_list.append(temp_edge)
G.add_edges_from(edge_list)


In [8]:
##successful
import networkx as nx

G=nx.Graph()
for temp_str in train_data:
  temp=temp_str.split('\t')
  temp_list=[int(i) for i in temp]
  temp_edges=[(temp_list[0], temp_list[i]) for i in range(1, len(temp_list))]
  G.add_edges_from(temp_edges)

In [12]:
print('number of nodes',G.number_of_nodes())
print('number of edges',G.number_of_edges())

number of nodes 4867136
number of edges 23416061


In [11]:
## Get undirected_graph
edge_list = []
import networkx as nx
SG = nx.Graph()
for temp_str in train_data[0:2]:
  temp=temp_str.split('\t')
  for i in range(1, len(temp)):
    temp_edge = (int(temp[0]), int(temp[i]))
    edge_list.append(temp_edge)
SG.add_edges_from(edge_list)

In [ ]:
import matplotlib.pyplot as plt
nx.draw(SG)

In [ ]:
print("Generating community......")
from networkx.algorithms import community

comms = list(community.asyn_fluidc(G,100))
print("Size of communities:"+str(len(comms)))

print("Adding community attribute......")
count=0
for node in G.nodes():
    if(count%100==0):
        print(count)
    count+=1
    for i in range(len(comms)):
        if node in comms[i]:
            G.nodes[node]['community'] = i

## features

In [10]:
def get_features(data_list):
    features = []
    count = 0
    print("Generating " + "data_list" + " features......")
    for temp_data in data_list:
      if (count % 100 == 0):
        print(count)
      count += 1
      feature = []
      try:
        preds = nx.resource_allocation_index(G, [temp_data])
        for u, v, p in preds:
          feature.append(p)
        
        preds = nx.jaccard_coefficient(G, [temp_data])
        for u, v, p in preds:
          feature.append(p)

            # preds = nx.adamic_adar_index(G, [temp_data])
            # for u, v, p in preds:
            #     feature.append(p)

            # preds = nx.preferential_attachment(G, [temp_data])
            # for u, v, p in preds:
            #     feature.append(p)

            # preds = nx.cn_soundarajan_hopcroft(G, [temp_data])
            # for u, v, p in preds:
            #     feature.append(p)

            # preds = nx.ra_index_soundarajan_hopcroft(G, [temp_data])
            # for u, v, p in preds:
            #     feature.append(p)

            # preds = nx.within_inter_cluster(G, [temp_data])
            # for u, v, p in preds:
            #     feature.append(p)          
        
        feature.append(1)  # label=1
        
      except:
        print("one error at: "+str(count))
        pass
      
      features.append(feature)
    print("positive features: "+str(len(features)))
    return features

## Combine

In [ ]:
a = get_features(positive_train)
#train_features = get_features(positive_train) + get_features(negative_train)

In [ ]:
print(a)

In [ ]:
test_features = get_features(test_edges)

## Save features

In [ ]:
import csv

with open("train_data.csv","w",newline="") as csvfile:
  writer=csv.writer(csvfile)
  writer.writerow(["RA","JC","Label"])
  #writer.writerow(["RA","JC","AA","PA","CSH","RSH","WIC","Label"])
  writer.writerows(train_features)

In [ ]:
with open("test.csv","w",newline="") as csvfile:
  writer=csv.writer(csvfile)
  writer.writerow(["RA","JC"])
  #writer.writerow(["RA","JC","AA","PA","CSH","RSH","WIC"])
  writer.writerows(test_features)